In [1]:
main_path = '../course-material-time-series-forecasting-product'
nyc_data = main_path+str("/nyc_data.csv")
print(nyc_data)

../course-material-time-series-forecasting-product/nyc_data.csv


In [2]:
# Libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid


In [3]:
# load the data
# YYYY-MM-DD
df = pd.read_csv(nyc_data, index_col=0, parse_dates=True)
df.head() 

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [4]:
# Rename variable 
df = df.rename(columns={'Demand': 'y'})
df.head()

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


# Prepare for LSTM 

#### 1. Creating time series object

In [5]:
from darts.timeseries import TimeSeries
series = TimeSeries.from_series(df['y'])
covariates = TimeSeries.from_dataframe(df.iloc[:, 1:]) # everything that's not y

In [6]:
# Seasonality / trend
from darts.utils.timeseries_generation import datetime_attribute_timeseries

# Year (time series object)
year_series = datetime_attribute_timeseries(pd.date_range(
    start=series.start_time(),
    freq=series.freq_str,
    periods=df.shape[0]),
    attribute='year',
    one_hot=False
)

# Month
month_series = datetime_attribute_timeseries(
    year_series, attribute='month', one_hot=True)

# Weekday
weekday_series = datetime_attribute_timeseries(
    year_series, attribute='weekday', one_hot=True)

In [7]:
# Scaling 
from darts.dataprocessing.transformers import Scaler

transformer1 = Scaler()
transformer2 = Scaler()

# Year series and non-binary data must be scaled

In [8]:
# Scaling Y
y_transformed = transformer1.fit_transform(series)

# Stacking the covariates
covariates = covariates.stack(year_series)

# Scaling the covariates
covariates_transformed = transformer2.fit_transform(covariates)

In [9]:
# Stack the seasonal variables
covariates_transformed = covariates_transformed.stack(month_series)
covariates_transformed = covariates_transformed.stack(weekday_series)

# LSTM Model

In [10]:
# Build the LSTM model (https://unit8co.github.io/darts/generated_api/darts.models.forecasting.rnn_model.html) 
from darts.models import RNNModel
model = RNNModel(
    model='LSTM',
    hidden_dim=20,
    n_rnn_layers=2,
    dropout=0.2,
    n_epochs=20,
    optimizer_kwargs={'lr': 1e-3},
    random_state=1502,
    training_length=20,
    input_chunk_length=15,
    # pl_trainer_kwargs={"accelerator": "cpu"} # for CPU,
    # pl_trainer_kwargs={'accelerator': 'gpu', 'devices': [0]}, #to use only GPU i (i must be an integer)
    pl_trainer_kwargs={"accelerator": "gpu", "devices": -1, "auto_select_gpus": True}
)

In [11]:
model.fit(y_transformed, future_covariates=covariates_transformed, verbose=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 7.2 K 
4 | V             | Linear           | 21    
---------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


RNNModel(model=LSTM, hidden_dim=20, n_rnn_layers=2, dropout=0.2, training_length=20, n_epochs=20, optimizer_kwargs={'lr': 0.001}, random_state=1502, input_chunk_length=15, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': [0]})

## Cross-Validation

In [12]:
# PS: To solve the ImportError: IProgress not found, you need to update Jupyter and ipywidgets by running the command %pip install --upgrade jupyter ipywidgets in a code cell.

# CV 
cv = model.historical_forecasts(
    series=y_transformed,
    future_covariates=covariates_transformed,
    start= df.shape[0]-180,
    forecast_horizon=31,
    stride=16,
    retrain=True,
    last_points_only=False,
    verbose=True,
)

`enable_optimization=True` is ignored because `retrain` is not `False` or `0`.To hide this warning, set `show_warnings=False` or `enable_optimization=False`.
`enable_optimization=True` is ignored because `forecast_horizon > model.output_chunk_length`.To hide this warning, set `show_warnings=False` or `enable_optimization=False`.


  0%|          | 0/10 [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 7.2 K 
4 | V             | Linear           | 21    
---------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 7.2 K 
4 | V             | Linear           | 21    
---------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [28]:
from sklearn.metrics import mean_squared_error

# Store the results 
rmse_cv = []

# Compute the RMSE for the CV
for i in range(len(cv)): # we have a lot of cv

    # We need to inverse transform. Apply TimeSeries and transform it to Pandas Series. We are going to compare it with the actual values
    predictions = TimeSeries.pd_series(transformer1.inverse_transform(cv[i]))

    # Actual values
    start = predictions.index.min();
    end = predictions.index.max();
    actuals = df.y[start:end];

    # compute the error
    error_cv = mean_squared_error(actuals, predictions, squared=False);
    
    # save the error
    rmse_cv.append(error_cv);

print(f"The RMSE is {np.mean(rmse_cv)}")

/home/renan/.python_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/renan/.python_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/renan/.python_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/renan/.python_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calcula

The RMSE is 84.00989157956035


/home/renan/.python_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/renan/.python_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Parameter tuning

In [31]:
# Grid 
param_grid = {
    'n_rnn_layers' : [1,2],
    'hidden_dim' : [10,20],
    'dropout' : [0.1, 0.2],
    'n_epochs': [10,20],
    'lr' : [1e-3],
    'training_length' : [20],
    'input_chunk_length' : [15]
}
grid = ParameterGrid(param_grid)
len(list(grid))

16

In [34]:
# Parameter tuning 
rmse = []

# Loop 
for params in grid: 
    
    # Build the model 
    model = RNNModel(
        model='LSTM',
        hidden_dim=params['hidden_dim'],
        n_rnn_layers=params['n_rnn_layers'],
        dropout=params['dropout'],
        n_epochs=params['n_epochs'],
        optimizer_kwargs={'lr': 1e-3},
        random_state=1502,
        training_length=20,
        input_chunk_length=15,
        pl_trainer_kwargs={"accelerator": "gpu", "devices": -1}
    )

    model.fit(y_transformed, future_covariates=covariates_transformed, verbose=True)

    # CV 
    cv = model.historical_forecasts(
        series=y_transformed,
        future_covariates=covariates_transformed,
        start= df.shape[0]-180,
        forecast_horizon=31,
        stride=16,
        retrain=True,
        last_points_only=False,
        verbose=True,
    )

    # Measure and store the error

    # Store the results 
    rmse_cv = []

    # Compute the RMSE for the CV
    for i in range(len(cv)): # we have a lot of cv

        # We need to inverse transform. Apply TimeSeries and transform it to Pandas Series. We are going to compare it with the actual values
        predictions = TimeSeries.pd_series(transformer1.inverse_transform(cv[i]))

        # Actual values
        start = predictions.index.min();
        end = predictions.index.max();
        actuals = df.y[start:end];

        # compute the error
        error_cv = mean_squared_error(actuals, predictions, squared=False);
        
        # save the error
        rmse_cv.append(error_cv);

        error = np.mean(rmse_cv)
    rmse.append(error)

/home/renan/.python_env/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 1.5 K 
4 | V             | Linear           | 11    
---------------------------------------------------
1.5 K     Trainable params
0         Non-trainable params
1.5 K     Total params
0.006     Total estimated mo

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
`enable_optimization=True` is ignored because `retrain` is not `False` or `0`.To hide this warning, set `show_warnings=False` or `enable_optimization=False`.
`enable_optimization=True` is ignored because `forecast_horizon > model.output_chunk_length`.To hide this warning, set `show_warnings=False` or `enable_optimization=False`.


  0%|          | 0/10 [00:00<?, ?it/s]

/home/renan/.python_env/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 1.5 K 
4 | V             | Linear           | 11    
---------------------------------------------------
1.5 K     Trainable params
0         Non-trainable params
1.5 K     Total params
0.006     Total estimated mo

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 7.2 K 
4 | V             | Linear           | 21    
---------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

/home/renan/.python_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/renan/.python_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/renan/.python_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/renan/.python_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calcula

In [ ]:
# Parameter Tuning outcome
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

# Exporting the tuned parameters
best_params = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()
best_params.to_csv("./lstm_best_params.csv")

In [ ]:
# Isolate the params
n_rnn_layers = int(best_params.iloc['n_rnn_layers'])
hidden_dim = int(best_params.iloc['hidden_dim'])
dropout = float(best_params.iloc['dropout'])

# n_epochs = int(best_params.iloc['n_epochs'])
# lr = float(best_params.iloc['lr'])
# training_length = int(best_params.iloc['training_length'])

### Parameter Tuning part 2

In [ ]:
# Grid 
param_grid = {
    'n_rnn_layers' : [n_rnn_layers],
    'hidden_dim' : [hidden_dim],
    'dropout' : [dropout],
    'n_epochs': [10,20],
    'lr' : [1e-3, 3e-3],
    'training_length' : [20, 30],
    'input_chunk_length' : [15, 20]
}
grid = ParameterGrid(param_grid)
len(list(grid))

In [ ]:
# Parameter tuning 
rmse = []

# Loop 
for params in grid: 
    
    # Build the model 
    model = RNNModel(
        model='LSTM',
        
        hidden_dim=params['hidden_dim'],
        n_rnn_layers=params['n_rnn_layers'],
        dropout=params['dropout'],
        n_epochs=params['n_epochs'],

        optimizer_kwargs=params['lr'],
        training_length=params['training_length'],
        input_chunk_length=params['input_chunk_length'],
        
        random_state=1502,
        pl_trainer_kwargs={"accelerator": "gpu", "devices": -1}
    )

    model.fit(y_transformed, future_covariates=covariates_transformed, verbose=True)

    # CV 
    cv = model.historical_forecasts(
        series=y_transformed,
        future_covariates=covariates_transformed,
        start= df.shape[0]-180,
        forecast_horizon=31,
        stride=16,
        retrain=True,
        last_points_only=False,
        verbose=True,
    )

    # Measure and store the error

    # Store the results 
    rmse_cv = []

    # Compute the RMSE for the CV
    for i in range(len(cv)): # we have a lot of cv

        # We need to inverse transform. Apply TimeSeries and transform it to Pandas Series. We are going to compare it with the actual values
        predictions = TimeSeries.pd_series(transformer1.inverse_transform(cv[i]))

        # Actual values
        start = predictions.index.min();
        end = predictions.index.max();
        actuals = df.y[start:end];

        # compute the error
        error_cv = mean_squared_error(actuals, predictions, squared=False);
        
        # save the error
        rmse_cv.append(error_cv);

        error = np.mean(rmse_cv)
    rmse.append(error)

In [ ]:
# Parameter Tuning outcome
tuning_results2 = pd.DataFrame(grid)
tuning_results2['rmse'] = rmse
tuning_results2

# Exporting the tuned parameters
best_params2 = tuning_results2[tuning_results2.rmse == tuning_results2.rmse.min()].transpose()
best_params2.to_csv("./lstm_best_params2.csv")